# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

- artist: The name of the artist associated with the song being played.
- auth: The authentication status of the user, indicating whether the user is logged in or logged out.
- firstName: The first name of the user.
- gender: The gender of the user.
- itemInSession: The index of the item within the user's session.
- lastName: The last name of the user.
- length: The duration of the song being played.
- level: The subscription level of the user (free or paid).
- location: The location of the user.
- method: The HTTP method used for the request (GET or PUT).
- page: The specific page or action the user performed (e.g., "NextSong", "Thumbs Up", "Add to Playlist").
- registration: The timestamp when the user registered.
- sessionId: The ID of the user's session.
- song: The title of the song being played.
- status: The HTTP status code of the response.
- ts: The timestamp of the event.
- userAgent: The user agent string of the user's browser or client.
- userId: The unique identifier of the user.

**Objective** to define a model the predict the users' potential negative behaviour - cancel, downgrade - based on the users basic json info

In [33]:
# import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, count, sum as sql_sum, avg, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
# create a Spark session
spark = SparkSession.builder \
    .appName("Sparkify Data Analysis") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [35]:
%ls

mini_sparkify_event_data.json  Sparkify.ipynb  Sparkify-zh.ipynb


In [36]:
file_path = "mini_sparkify_event_data.json"
# read the data via spark
sc = spark.read.json(file_path)

In [37]:
sc.printSchema()
sc.show(5)

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|     

In [38]:
sc.head(5)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [39]:
sc.select([count(when(col(c).isNull(), c)).alias(c) for c in sc.columns]).show(2)

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [40]:
# convert it to pandas dataframe for a better overview
df = sc.toPandas()

In [41]:
df.isnull().sum()

artist           58392
auth                 0
firstName         8346
gender            8346
itemInSession        0
lastName          8346
length           58392
level                0
location          8346
method               0
page                 0
registration      8346
sessionId            0
song             58392
status               0
ts                   0
userAgent         8346
userId               0
dtype: int64

> From the above columns and the number of null values in each, the parameters that may determine the user behaviour may cover: gender, itemInSession, level, location, method, page, sessionId, status, ts, and userId. Using these parameters can minimise the null value to be dropped and give the maximum parameters for the model training

In [42]:
df.sample(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
224990,Metallica,Logged In,Jayden,M,17,Santos,433.08363,paid,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.533813e+12,200,The Four Horsemen,200,1542800736000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",100018
76265,None,Logged In,Colin,M,84,Larson,NaN,paid,"Dallas-Fort Worth-Arlington, TX",PUT,Thumbs Up,1.537982e+12,1049,None,307,1540275380000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) G...,100
17450,LeAnn Rimes,Logged In,Grant,M,84,Flores,214.59546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.538121e+12,498,Can't Fight The Moonlight (Theme From 'Coyote ...,200,1538851210000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",142
259837,Caparezza,Logged In,Jade,F,138,Stewart,230.13832,paid,"Cookeville, TN",PUT,NextSong,1.535660e+12,171,La Legge Dell'Ortica,200,1539827986000,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1 like M...",300008
219534,Rise Against,Logged In,London,F,47,Diaz,200.12363,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.532626e+12,123,Swing Life Away,200,1539933904000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100025


In [43]:
df.page.unique()

array(['NextSong', 'Add to Playlist', 'Roll Advert', 'Thumbs Up',
       'Downgrade', 'Thumbs Down', 'Home', 'Logout', 'Help', 'Login',
       'Upgrade', 'Add Friend', 'About', 'Settings', 'Submit Upgrade',
       'Submit Downgrade', 'Error', 'Save Settings', 'Cancel',
       'Cancellation Confirmation', 'Register', 'Submit Registration'], dtype=object)

In [44]:
df.describe()

,itemInSession,length,registration,sessionId,status,ts
count,286500.000000,228108.000000,2.781540e+05,286500.000000,286500.000000,2.865000e+05
mean,114.414213,249.117182,1.535359e+12,1041.526555,210.054597,1.540957e+12
std,129.767262,99.235179,3.291322e+09,726.776263,31.505078,1.507544e+09
min,0.000000,0.783220,1.521381e+12,1.000000,200.000000,1.538352e+12
25%,27.000000,199.888530,1.533522e+12,336.000000,200.000000,1.539697e+12
50%,70.000000,234.500770,1.536664e+12,1013.000000,200.000000,1.540932e+12
75%,156.250000,277.158730,1.537672e+12,1675.000000,200.000000,1.542266e+12
max,1321.000000,3024.665670,1.543247e+12,2474.000000,404.000000,1.543799e+12


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [45]:
sc.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

In [46]:
# rename the column names
from pyspark.sql.types import StringType
to_lower_coln = udf(lambda x: x.lower(), StringType())

In [47]:
for col in sc.columns:
    sc.withColumn(col, to_lower_coln(col))

In [48]:
# select the relevant columns
coln_selected = ['gender', 'itemInSession', 'level', 'location', 'method', 'page', 'sessionId', 'status', 'ts', 'userId']
sc_sl = sc.select(coln_selected)

In [51]:
# drop na and duplicated
sc_cl = sc_sl.dropna()
sc_cl = sc_cl.dropDuplicates()

In [56]:
# add a column of churn
to_label_churn = udf(lambda x: 2 if x == 'Cancellation Confirmation' else 1 if x == 'Downgrade' else 0)

In [57]:
sc_cl = sc_cl.withColumn('churn', to_label_churn(sc_cl.page))

In [59]:
# convert time
import datetime
get_year = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). year)
get_month = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). month)
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). day)
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)

In [60]:
sc_cl = sc_cl.withColumn('year', get_year(sc_cl.ts))
sc_cl = sc_cl.withColumn('month', get_month(sc_cl.ts))
sc_cl = sc_cl.withColumn('day', get_day(sc_cl.ts))
sc_cl = sc_cl.withColumn('hour', get_hour(sc_cl.ts))

In [62]:
sc_cl = sc_cl.drop('ts')

In [66]:
# select on the state as the location
to_extract_state = udf(lambda s: s.split(',')[1].strip() if ',' in s else s)

In [67]:
sc_cl = sc_cl.withColumn('state', to_extract_state(sc_cl.location))

In [83]:
from pyspark.sql.functions import split, explode
sc_cl2 = sc_cl.withColumn('states', split(sc_cl.state, "-"))
sc_cl2 = sc_cl2.withColumn('state', explode(sc_cl2.states))
sc_cl = sc_cl2.drop('states','location')

In [88]:
sc_cl2.toPandas().month.unique()

array(['10', '11', '12'], dtype=object)

In [86]:
sc_cl.show(5)

+------+-------------+-----+------+--------------+---------+------+------+-----+----+-----+---+----+-----+
|gender|itemInSession|level|method|          page|sessionId|status|userId|churn|year|month|day|hour|state|
+------+-------------+-----+------+--------------+---------+------+------+-----+----+-----+---+----+-----+
|     M|           17| free|   PUT|      NextSong|      240|   200|     4|    0|2018|   10|  1|   2|   MD|
|     F|           18| free|   PUT|     Thumbs Up|      187|   307|    78|    0|2018|   10|  1|   2|   TX|
|     M|           11| free|   PUT|Submit Upgrade|      141|   307|   142|    0|2018|   10|  1|   6|   NY|
|     M|           11| free|   PUT|Submit Upgrade|      141|   307|   142|    0|2018|   10|  1|   6|   NJ|
|     M|           11| free|   PUT|Submit Upgrade|      141|   307|   142|    0|2018|   10|  1|   6|   PA|
+------+-------------+-----+------+--------------+---------+------+------+-----+----+-----+---+----+-----+
only showing top 5 rows



> Now the data is wrangled

In [ ]:
# Aggregate data
spd_aggregated = spd_transformed.groupBy("userId").agg(
    count("*").alias("count"),
    avg("length").alias("average_length"),
    sum("length").alias("total_length")
)
spd_aggregated.show()


In [ ]:
sc_ind.show(5)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
# indexing the strings
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
for col in sc_cl.columns:
    indexer = StringIndexer(inputCol=col, outputCol=col + "_index")
    sc_ind = indexer.fit(sc_cl).transform(sc_cl)
    sc_ind = sc_ind.drop(col).withColumnRenamed(col + "_index", col)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.